In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings 
warnings.filterwarnings('ignore')

import missingno as msno 

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

%matplotlib inline

In [80]:
df_train = pd.read_csv('./titanic/train.csv')
df_test = pd.read_csv('./titanic/test.csv')

df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 #1은 본인
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1


df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i>0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i>0 else 0)

df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,1.981001,NaN,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,4.266662,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,2.070022,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,3.972177,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,2.085672,NaN,S,1


In [81]:
#null로 된 값 채우기
#title + statistics를 사용해서 null을 채운다.(Miss, Mrr, Mrs)
#pandas series 중 data를 string으로 바꿔주고 여기서 정규표현식을 적용하는 extract method가 있다. 이를 적용
df_train['Initial']= df_train.Name.str.extract('([A-Za-z]+)\.') #salutations을 추출
df_test['Initial']= df_test.Name.str.extract('([A-Za-z]+)\.') #salutations을 추출

#치환
#남자 여자가쓰는 inital을 구분하여 볼 수 있도록

df_train['Initial'].replace(['Mile','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
                           ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)
df_test['Initial'].replace(['Mile','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
                           ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap='summer_r')

Initial,Master,Miss,Mlle,Mr,Mrs,Other
Sex,,,,,,
female,0,184,2,1,127,0
male,40,0,0,528,0,9


In [82]:

#남녀구분을 위처럼 둔뒤, AGE의 null값을 채울 차례.
#pandas df를 다룰 땐, boolean array를 이용하라? 0or1의 값으로 된 애들을 필터하는것?

df_train.loc[(df_train.Age.isnull())&(df_train.Initial == 'Mr'),'Age'] = 33
#df_train에서 Age에 null값이고 Initial이 Mr인 Age의 value를 33으로 한다.
df_train.loc[(df_train.Age.isnull())&(df_train.Initial == 'Mrs'),'Age'] = 36
#df_train에서 Age에 null값이고 Initial이 Mrs인 Age의 value를 36으로 한다.
df_train.loc[(df_train.Age.isnull())&(df_train.Initial == 'Master'),'Age'] = 5
df_train.loc[(df_train.Age.isnull())&(df_train.Initial == 'Miss'),'Age'] = 22
df_train.loc[(df_train.Age.isnull())&(df_train.Initial == 'Other'),'Age'] = 46


#df_test는 학습에 넣을 데이터
df_test.loc[(df_test.Age.isnull())&(df_test.Initial == 'Mr'),'Age'] = 33
#df_test Age에 null값이고 Initial이 Mr인 Age의 value를 33으로 한다.
df_test.loc[(df_test.Age.isnull())&(df_test.Initial == 'Mrs'),'Age'] = 36
#df_test Age에 null값이고 Initial이 Mrs인 Age의 value를 36으로 한다.
df_test.loc[(df_test.Age.isnull())&(df_test.Initial == 'Master'),'Age'] = 5
df_test.loc[(df_test.Age.isnull())&(df_test.Initial == 'Miss'),'Age'] = 22
df_test.loc[(df_test.Age.isnull())&(df_test.Initial == 'Other'),'Age'] = 46

df_train.groupby('Initial').mean() #활용
df_test.groupby('Initial').mean()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,FamilySize
Initial,,,,,,,
Master,1123.380952,2.714286,6.948095,1.571429,1.380952,3.112730,3.952381
Miss,1100.240506,2.443038,21.817595,0.544304,0.392405,2.927526,1.936709
Mr,1102.471074,2.309917,32.351240,0.272727,0.177686,2.729754,1.450413
Mrs,1090.097222,1.833333,38.500000,0.597222,0.833333,3.566570,2.430556
Other,1053.500000,1.500000,42.750000,0.500000,0.250000,3.650053,1.750000


In [83]:
#Embarked는 s에 가장 많은 탑승객이 있으므로 2개는 s로 바꿔준다.
df_train['Embarked'].fillna('S', inplace=True)
df_test['Embarked'].fillna('S', inplace=True)

In [84]:
#Age 카테고리
#apply
def category_age(x) :
    if x < 10 :
        return 0
    elif x < 20 :
        return 1
    elif x < 30 : 
        return 2
    elif x < 40:
        return 3
    elif x < 50:
        return 4
    elif x < 60:
        return 5
    elif x < 70:
        return 6
    else:
        return 7  
df_train['Age_cat'] = df_train['Age'].apply(category_age)
df_test['Age_cat'] = df_test['Age'].apply(category_age)

In [85]:
#Mrs, Ms 와 같은 word를 컴퓨터가 인식할 수 있는 값으로 맵핑
df_train['Initial'] = df_train['Initial'].map({
    'Master' : 0,
    'Miss' : 1,
    'Mr' : 2,
    'Mrs' : 3,
    'Other': 4
})

df_test['Initial'] = df_test['Initial'].map({
    'Master' : 0,
    'Miss' : 1,
    'Mr' : 2,
    'Mrs' : 3,
    'Other': 4
})

df_train['Embarked'] = df_train['Embarked'].map({
    'C' : 0,
    'Q' : 1,
    'S' : 2
})

df_test['Embarked'] = df_test['Embarked'].map({
    'C' : 0,
    'Q' : 1,
    'S' : 2
})

#Sex도 맵핑
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})

df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1})

df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Initial,Age_cat
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,1.981001,NaN,2,2,2.0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,4.266662,C85,0,2,3.0,3
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,2.070022,NaN,2,1,1.0,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,3.972177,C123,2,2,3.0,3
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,2.085672,NaN,2,1,2.0,3


In [86]:
#one-got : pandas의 get_dummies 이용

df_train = pd.get_dummies(df_train, columns=['Initial'], prefix='Initial')
#구분을 두는 접두사, prefix. prfix로 구분지므로 prefix에 해당하는 값은 1로 되어 표로  나타남.
#기존의 표에 컬럼이 추가 되는 것.
df_test = pd.get_dummies(df_test, columns=['Initial'], prefix='Initial')

In [87]:
df_train= pd.get_dummies(df_train, columns=['Embarked'], prefix='Embarked')
df_test= pd.get_dummies(df_test, columns=['Embarked'], prefix='Embarked')

In [88]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,FamilySize,Age_cat,Initial_0.0,Initial_1.0,Initial_2.0,Initial_3.0,Initial_4.0,Embarked_0,Embarked_1,Embarked_2
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,1.981001,...,2,2,0,0,1,0,0,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,4.266662,...,2,3,0,0,0,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,2.070022,...,1,2,0,1,0,0,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,3.972177,...,2,3,0,0,0,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,2.085672,...,1,3,0,0,1,0,0,0,0,1


In [89]:
# #label encoder
# from sklearn import preprocessing

# y_true_label_encoder = preprocessing.LabelEncoder()
# y_true_encoded = y_true_label_encoder.fit_transform(df_train['Ticket'])
# y_true_encoded_test = y_true_label_encoder.fit_transform(df_test['Ticket'])
# df_train['ticket_encode'] = y_true_encoded
# df_test['ticket_encode'] = y_true_encoded_test

In [90]:
# #category화
# t={'str':[],'int':[]}
# for i in df_train['Ticket'].values :
#     try :
#         num=int(i)
#         if num not in t['int'] :
#             t['int'].append(i)
#     except ValueError :
#         if i not in t['str'] :
#             t['str'].append(i)
# m=df_train[df_train['Ticket'].isin(t['int'])]
# n=df_train[df_train['Ticket'].isin(t['str'])]
# s={'1':[],'2':[],'3':[]}
# for i in range(1,4) :
#     for j in sorted(m[m['Pclass'].isin([i])]['Ticket']) :
#         s[str(i)].append(j)
#     for j in sorted(n[n['Pclass'].isin([i])]['Ticket']) :
#         s[str(i)].append(j)
# class_ticket=[]
# for ticket in df_train['Ticket'] :
#     for j in range(1,4) :
#         if ticket in s[str(j)] :
# #             print(f'{ticket}은 Pclass {j}등급에 탔음')
#             class_ticket.append(j-1)
# print(class_ticket)

# t_test={'str':[],'int':[]}
# for i in df_test['Ticket'].values :
#     try :
#         num=int(i)
#         if num not in t['int'] :
#             t['int'].append(i)
#     except ValueError :
#         if i not in t['str'] :
#             t['str'].append(i)
# m=df_test[df_test['Ticket'].isin(t['int'])]
# n=df_test[df_test['Ticket'].isin(t['str'])]
# s={'1':[],'2':[],'3':[]}
# for i in range(1,4) :
#     for j in sorted(m[m['Pclass'].isin([i])]['Ticket']) :
#         s[str(i)].append(j)
#     for j in sorted(n[n['Pclass'].isin([i])]['Ticket']) :
#         s[str(i)].append(j)
# class_ticket_test=[]
# for ticket in df_test['Ticket'] :
#     for j in range(1,4) :
#         if ticket in s[str(j)] :
# #             print(f'{ticket}은 Pclass {j}등급에 탔음')
#             class_ticket_test.append(j-1)
# print(class_ticket_test)

# df_train['Ticket_encode'] = class_ticket
# df_test['Ticket_encode'] = class_ticket_test

In [91]:
# #문자/숫자 띠로
# import re
# from sklearn import preprocessing
# y_true_label_encoder = preprocessing.LabelEncoder()

# class_ticket=[]
# encoding_ticket=[]
# num_ticket=[]
# for ticket in df_train['Ticket'] :
#     a,b,c='','',''
#     for t in ticket.split() :
#         if t[0].isalpha() :
#             a="".join(re.findall('[A-Z]',t))
#             b="".join(re.findall('[0-9]',t))
#         elif t[0].isdigit() :
#             c=t
#     encoding_ticket.append(a+b)
#     num_ticket.append(c)
# encoded = y_true_label_encoder.fit_transform(encoding_ticket)
# for i in range(len(encoded)) :
#     ticket_encode=str(encoded[i])
#     if num_ticket[i] == '' :
#         pass
#     else :
#         ticket_encode+=(num_ticket[i])
#     class_ticket.append(int(ticket_encode))
# print(class_ticket)

# class_ticket_test=[]
# encoding_ticket_test=[]
# num_ticket_test=[]
# for ticket in df_test['Ticket'] :
#     a,b,c='','',''
#     for t in ticket.split() :
#         if t[0].isalpha() :
#             a="".join(re.findall('[A-Z]',t))
#             b="".join(re.findall('[0-9]',t))
#         elif t[0].isdigit() :
#             c=t
#     encoding_ticket_test.append(a+b)
#     num_ticket_test.append(c)
# encoded_test = y_true_label_encoder.fit_transform(encoding_ticket_test)
# for i in range(len(encoded_test)) :
#     ticket_encode_test=str(encoded_test[i])
#     if num_ticket_test[i] == '' :
#         pass
#     else :
#         ticket_encode_test+=(num_ticket_test[i])
#     class_ticket_test.append(int(ticket_encode_test))
# print(class_ticket_test)

In [93]:
#label인코더랑 문자 통합한 후 진행
import re
from sklearn import preprocessing
y_true_label_encoder = preprocessing.LabelEncoder()

class_ticket=[]
encoding_ticket=[]
num_ticket=[]
for ticket in df_train['Ticket'] :
    a,b,c='','',''
    for t in ticket.split() :
        if t[0].isalpha() :
            a="".join(re.findall('[A-Z]',t))
            b="".join(re.findall('[0-9]',t))
        elif t[0].isdigit() :
            c=t
    num_ticket.append(a+b+c)
class_ticket= y_true_label_encoder.fit_transform(num_ticket)
print(class_ticket)


class_ticket_test=[]
encoding_ticket_test=[]
num_ticket_test=[]
for ticket in df_test['Ticket'] :
    a,b,c='','',''
    for t in ticket.split() :
        if t[0].isalpha() :
            a="".join(re.findall('[A-Z]',t))
            b="".join(re.findall('[0-9]',t))
        elif t[0].isdigit() :
            c=t
    num_ticket_test.append(a+b+c)
class_ticket_test= y_true_label_encoder.fit_transform(num_ticket_test)
print(class_ticket_test)

[524 596 656  49 472 275  85 395 344 132 616  38 528 333 413 153 480 151
 301 184 139 152 278  42 395 329 179  95 283 362 598 586 288 553 600  43
 202 529 302 186 507  53 391 623 282 619 462  76 190 377 249 516 587 239
  29  94 559 215 565 549 197  32 457 337 601 189 557 653 248 270 561 549
 631 204  80 347 355 473 158 432 304 311 280  25 632 247 677 649  95 297
 298 341 679 551 428 475 607 612 117 150 384 361 417 505 245 354 296 254
 388 469   2 193 274 491 176 670 460  53 585 333 631 519 132 219 417 186
 463 542 196 318 380 640 534 111 556 627  55  49 504 592 203 332 655 438
 118 562 405 677 113 147 633  36 521 539 570 595 418 652 258 552 470 563
 323 265 249 423  28 337 576  80   5 480 344 663  90 412 488 593 169 575
 552 626 329 114 271  34 459  11 435 386 122 227 406 113 604 586 452 494
 461 160 306 552 242 177 536 316 247 216 445  17 642 572 527 163 443 416
 657 142  63 673 651 105 102 375  93 615 671 526 125 488 457 322 128 329
 558 678 198 560 231 622 193 444 675 662 213  92 32

In [94]:
df_train['ticket_encode']=class_ticket
df_test['ticket_encode']=class_ticket_test

In [95]:
# df_train= pd.get_dummies(df_train, columns=['Ticket_encode'], prefix='Ticket_encode')
# df_test= pd.get_dummies(df_test, columns=['Ticket_encode'], prefix='Ticket_encode')

In [96]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Age_cat,Initial_0.0,Initial_1.0,Initial_2.0,Initial_3.0,Initial_4.0,Embarked_0,Embarked_1,Embarked_2,ticket_encode
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,1.981001,...,2,0,0,1,0,0,0,0,1,524
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,4.266662,...,3,0,0,0,1,0,1,0,0,596
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,2.070022,...,2,0,1,0,0,0,0,0,1,656
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,3.972177,...,3,0,0,0,1,0,0,0,1,49
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,2.085672,...,3,0,0,1,0,0,0,0,1,472


In [97]:
#모델에 필요한 컬럼만 남기고 나머지 컬럼 지우기
df_train.drop(['PassengerId','Age', 'Name','SibSp','Parch','Ticket','Cabin'],axis=1,inplace=True)

df_test.drop(['PassengerId', 'Age', 'Name','SibSp','Parch','Ticket','Cabin'],axis=1,inplace=True)

df_train.head()

,Survived,Pclass,Sex,Fare,FamilySize,Age_cat,Initial_0.0,Initial_1.0,Initial_2.0,Initial_3.0,Initial_4.0,Embarked_0,Embarked_1,Embarked_2,ticket_encode
0,0,3,1,1.981001,2,2,0,0,1,0,0,0,0,1,524
1,1,1,0,4.266662,2,3,0,0,0,1,0,1,0,0,596
2,1,3,0,2.070022,1,2,0,1,0,0,0,0,0,1,656
3,1,1,0,3.972177,2,3,0,0,0,1,0,0,0,1,49
4,0,3,1,2.085672,1,3,0,0,1,0,0,0,0,1,472


In [98]:
df_train.to_csv('./titanic/preprocessing.csv',index=False)

In [99]:
df_test.to_csv('./titanic/preprocessing_test.csv',index=False)